In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns',50)

In [2]:
# uploading the main datasets
keeper = pd.read_csv('2019-20_gk.csv')
outfield = pd.read_csv('2019-20_out.csv')
fpl = pd.read_csv('cleaned_players.csv')

In [3]:
keeper.columns

Index(['player', 'nationality', 'position', 'squad', 'age', 'birth_year',
       'games_gk', 'games_starts_gk', 'minutes_gk', 'goals_against_gk',
       'goals_against_per90_gk', 'shots_on_target_against', 'saves',
       'save_pct', 'wins_gk', 'draws_gk', 'losses_gk', 'clean_sheets',
       'clean_sheets_pct', 'pens_att_gk', 'pens_allowed', 'pens_saved',
       'pens_missed_gk', 'minutes_90s_gk', 'free_kick_goals_against_gk',
       'corner_kick_goals_against_gk', 'own_goals_against_gk', 'psxg_gk',
       'psnpxg_per_shot_on_target_against', 'psxg_net_gk', 'psxg_net_per90_gk',
       'passes_completed_launched_gk', 'passes_launched_gk',
       'passes_pct_launched_gk', 'passes_gk', 'passes_throws_gk',
       'pct_passes_launched_gk', 'passes_length_avg_gk', 'goal_kicks',
       'pct_goal_kicks_launched', 'goal_kick_length_avg', 'crosses_gk',
       'crosses_stopped_gk', 'crosses_stopped_pct_gk',
       'def_actions_outside_pen_area_gk',
       'def_actions_outside_pen_area_per90_gk', 

In [4]:
keeper.head(3)

player nationality position      squad  age  birth_year  \
0             Adrián      es ESP       GK  Liverpool   32        1987   
1            Alisson      br BRA       GK  Liverpool   26        1992   
2  Kepa Arrizabalaga      es ESP       GK    Chelsea   24        1994   

   games_gk  games_starts_gk  minutes_gk  goals_against_gk  \
0      11.0              9.0       875.0              10.0   
1      29.0             29.0      2543.0              23.0   
2      33.0             33.0      2970.0              47.0   

   goals_against_per90_gk  shots_on_target_against  saves  save_pct  wins_gk  \
0                    1.03                     28.0   17.0     0.643      9.0   
1                    0.81                     79.0   58.0     0.722     23.0   
2                    1.42                     99.0   55.0     0.545     18.0   

   draws_gk  losses_gk  clean_sheets  clean_sheets_pct  pens_att_gk  \
0       0.0        0.0           2.0              22.2          0.0   
1       3.0        3.0          13.0              44.8          1.0   
2       4.0       11.0           8.0              24.2          2.0   

   pens_allowed  pens_saved  pens_missed_gk  minutes_90s_gk  \
0           0.0         0.0             0.0             9.7   
1           1.0         0.0             0.0            28.3   
2           2.0         0.0             0.0            33.0   

   free_kick_goals_against_gk  corner_kick_goals_against_gk  \
0                         1.0                           1.0   
1                         0.0                           1.0   
2                         2.0                           8.0   

   own_goals_against_gk  psxg_gk  psnpxg_per_shot_on_target_against  \
0                   0.0      8.7                               0.31   
1                   1.0     22.8                               0.28   
2                   2.0     35.4                               0.35   

   psxg_net_gk  psxg_net_per90_gk  passes_completed_launched_gk  \
0         -1.3              -0.13                          60.0   
1          0.8               0.03                         138.0   
2         -9.6              -0.29                         143.0   

   passes_launched_gk  passes_pct_launched_gk  passes_gk  passes_throws_gk  \
0               142.0                    42.3      246.0              49.0   
1               255.0                    54.1      755.0             132.0   
2               334.0                    42.8      887.0             145.0   

   pct_passes_launched_gk  passes_length_avg_gk  goal_kicks  \
0                    44.3                  38.3        68.0   
1                    24.9                  31.7       170.0   
2                    28.2                  31.8       199.0   

   pct_goal_kicks_launched  goal_kick_length_avg  crosses_gk  \
0                     48.5                  40.4        91.0   
1                     39.4                  38.8       197.0   
2                     42.2                  39.1       236.0   

   crosses_stopped_gk  crosses_stopped_pct_gk  \
0                 8.0                     8.8   
1                16.0                     8.1   
2                17.0                     7.2   

   def_actions_outside_pen_area_gk  def_actions_outside_pen_area_per90_gk  \
0                              7.0                                   0.72   
1                             37.0                                   1.31   
2                             21.0                                   0.64   

   avg_distance_def_actions_gk  
0                         13.3  
1                         17.5  
2                         16.0

In [5]:
# cleaning the keepers dataset with required stats
keeper = keeper.loc[:,['player','squad','position','games_gk','minutes_gk','goals_against_per90_gk','save_pct',
                       'clean_sheets_pct','pens_allowed','psxg_net_gk']]
keeper

player            squad position  games_gk  minutes_gk  \
0              Adrián        Liverpool       GK      11.0       875.0   
1             Alisson        Liverpool       GK      29.0      2543.0   
2   Kepa Arrizabalaga          Chelsea       GK      33.0      2970.0   
3       Claudio Bravo  Manchester City       GK       4.0       347.0   
4     Willy Caballero          Chelsea       GK       5.0       450.0   
5     Martin Dúbravka    Newcastle Utd       GK      38.0      3420.0   
6             Ederson  Manchester City       GK      35.0      3071.0   
7    Łukasz Fabiański         West Ham       GK      25.0      2117.0   
8       Ralf Fährmann     Norwich City       GK       1.0        21.0   
9          Ben Foster          Watford       GK      38.0      3420.0   
10    Paulo Gazzaniga        Tottenham       GK      18.0      1613.0   
11       David de Gea   Manchester Utd       GK      38.0      3420.0   
12     Vicente Guaita   Crystal Palace       GK      35.0      3150.0   
13         Angus Gunn      Southampton       GK      10.0       900.0   
14         Tom Heaton      Aston Villa       GK      20.0      1794.0   
15     Dean Henderson    Sheffield Utd       GK      36.0      3240.0   
16    Wayne Hennessey   Crystal Palace       GK       3.0       270.0   
17           Tim Krul     Norwich City       GK      36.0      3240.0   
18         Bernd Leno          Arsenal       GK      30.0      2649.0   
19        Hugo Lloris        Tottenham       GK      21.0      1807.0   
20       David Martin         West Ham       GK       5.0       436.0   
21  Emiliano Martínez          Arsenal       GK       9.0       771.0   
22      Alex McCarthy      Southampton       GK      28.0      2520.0   
23   Michael McGovern     Norwich City       GK       2.0       159.0   
24        Simon Moore    Sheffield Utd       GK       2.0       180.0   
25       Ørjan Nyland      Aston Villa       GK       7.0       539.0   
26       Rui Patrício           Wolves       GK      38.0      3420.0   
27    Jordan Pickford          Everton       GK      38.0      3420.0   
28          Nick Pope          Burnley       GK      38.0      3420.0   
29     Aaron Ramsdale      Bournemouth       GK      37.0      3330.0   
30    Darren Randolph         West Ham       GK       2.0       180.0   
31         Pepe Reina      Aston Villa       GK      12.0      1080.0   
32            Roberto         West Ham       GK       8.0       687.0   
33        Mathew Ryan         Brighton       GK      38.0      3420.0   
34  Kasper Schmeichel   Leicester City       GK      38.0      3420.0   
35          Jed Steer      Aston Villa       GK       1.0         7.0   
36       Mark Travers      Bournemouth       GK       1.0        90.0   

    goals_against_per90_gk  save_pct  clean_sheets_pct  pens_allowed  \
0                     1.03     0.643              22.2           0.0   
1                     0.81     0.722              44.8           1.0   
2                     1.42     0.545              24.2           2.0   
3                     1.82     0.533              33.3           0.0   
4                     1.40     0.563              20.0           0.0   
5                     1.53     0.705              28.9           2.0   
6                     0.82     0.728              45.7           3.0   
7                     1.45     0.684              20.0           3.0   
8                     4.29     1.000               0.0           1.0   
9                     1.68     0.663              23.7           8.0   
10                    1.45     0.709              11.8           3.0   
11                    0.95     0.736              34.2           3.0   
12                    1.20     0.714              28.6           0.0   
13                    2.50     0.566              20.0           2.0   
14                    1.76     0.677              20.0           3.0   
15                    0.92     0.752              36.1           1.0   
16              

In [6]:
# removing relegated teams and their players
relegated = ['Bournemoth','Norwich City','Watford']
keeper = keeper[~keeper['squad'].isin(relegated)]
keeper.head()

player            squad position  games_gk  minutes_gk  \
0             Adrián        Liverpool       GK      11.0       875.0   
1            Alisson        Liverpool       GK      29.0      2543.0   
2  Kepa Arrizabalaga          Chelsea       GK      33.0      2970.0   
3      Claudio Bravo  Manchester City       GK       4.0       347.0   
4    Willy Caballero          Chelsea       GK       5.0       450.0   

   goals_against_per90_gk  save_pct  clean_sheets_pct  pens_allowed  \
0                    1.03     0.643              22.2           0.0   
1                    0.81     0.722              44.8           1.0   
2                    1.42     0.545              24.2           2.0   
3                    1.82     0.533              33.3           0.0   
4                    1.40     0.563              20.0           0.0   

   psxg_net_gk  
0         -1.3  
1          0.8  
2         -9.6  
3         -3.0  
4         -0.4

In [7]:
keeper.shape

(33, 10)

In [8]:
fpl_gk = fpl[fpl['element_type']=='GK'].loc[:,['first_name','second_name','total_points','now_cost']].reset_index(drop=True)
fpl_gk['name'] = fpl_gk['first_name'].str.cat(fpl_gk['second_name'],sep=' ')
fpl_gk = fpl_gk.drop(columns=['first_name','second_name'])
fpl_gk

total_points  now_cost                            name
0            114        50                      Bernd Leno
1             33        45               Emiliano Martínez
2             59        45                      Tom Heaton
3              1        40                       Jed Steer
4             15        40                    Ørjan Nyland
5            135        45                     Mathew Ryan
6              0        40                Christian Walton
7            170        55                       Nick Pope
8              0        40          Bailey Peacock-Farrell
9             13        50                 Willy Caballero
10            90        50               Kepa Arrizabalaga
11             7        45                 Wayne Hennessey
12           137        50                  Vicente Guaita
13             0        45                     Jonas Lössl
14           117        50                 Jordan Pickford
15             0        45                   João Virgínia
16             0        40         Fabricio Agosto Ramírez
17             0        45               Marcus Bettinelli
18             0        45                     Marek Rodák
19           156        55               Kasper Schmeichel
20             0        45                      Danny Ward
21             0        45        Francisco Casilla Cortés
22             0        40                    Kamil Miazek
23             0        45                   Illan Meslier
24            27        45  Adrián San Miguel del Castillo
25           122        60           Alisson Ramses Becker
26            11        45                   Claudio Bravo
27           133        60       Ederson Santana de Moraes
28             0        50                   Sergio Romero
29           143        55                    David de Gea
30           160        55                  Dean Henderson
31             0        50                     Karl Darlow
32           143        50                 Martin Dubravka
33             0        45                  Mark Gillespie
34             3        45                     Simon Moore
35             0        45                 Wes Foderingham
36           126        50                  Aaron Ramsdale
37           104        45                   Alex McCarthy
38            29        45                      Angus Gunn
39            98        55                     Hugo Lloris
40            58        45                 Paulo Gazzaniga
41             0        45                        Joe Hart
42             0        40                    David Button
43             0        45                   Sam Johnstone
44            20        40                    David Martin
45             3        45                 Darren Randolph
46            83        50                Lukasz Fabianski
47            18        45            Roberto Jimenez Gago
48             0        45                      John Ruddy
49           153        55   Rui Pedro dos Santos Patrício
50             0        45                   Matija Sarkic

In [9]:
keeper_cost = dict()
keeper_points = dict()
fpl_names = [n for n in fpl_gk['name']]
i=0
for kr in keeper.itertuples():
    for fplr in fpl_gk.itertuples():
        if re.search(kr[1],fplr[3]):
            keeper_cost[kr[1]] = fplr[2]/10
            keeper_points[kr[1]] = fplr[1]
            break
        elif kr[1]=='Martin Dúbravka':
            keeper_cost[kr[1]] = 5.0
            keeper_points[kr[1]] = 143
            break
        elif kr[1]=='Łukasz Fabiański':
            keeper_cost[kr[1]] = 5.0
            keeper_points[kr[1]] = 83
            break
        elif kr[1]=='Rui Patrício':
            keeper_cost[kr[1]] = 5.0
            keeper_points[kr[1]] = 83
            break
        else:
            keeper_cost[kr[1]] = 0
            keeper_points[kr[1]] =0
            
keeper['cost'] = keeper['player'].map(keeper_cost)
keeper['points'] = keeper['player'].map(keeper_points)
# mark travers and pepe reina left
left_gk = ['Pepe Reina','Mark Travers']
keeper = keeper[~keeper['player'].isin(left_gk)]
keeper

player            squad position  games_gk  minutes_gk  \
0              Adrián        Liverpool       GK      11.0       875.0   
1             Alisson        Liverpool       GK      29.0      2543.0   
2   Kepa Arrizabalaga          Chelsea       GK      33.0      2970.0   
3       Claudio Bravo  Manchester City       GK       4.0       347.0   
4     Willy Caballero          Chelsea       GK       5.0       450.0   
5     Martin Dúbravka    Newcastle Utd       GK      38.0      3420.0   
6             Ederson  Manchester City       GK      35.0      3071.0   
7    Łukasz Fabiański         West Ham       GK      25.0      2117.0   
10    Paulo Gazzaniga        Tottenham       GK      18.0      1613.0   
11       David de Gea   Manchester Utd       GK      38.0      3420.0   
12     Vicente Guaita   Crystal Palace       GK      35.0      3150.0   
13         Angus Gunn      Southampton       GK      10.0       900.0   
14         Tom Heaton      Aston Villa       GK      20.0      1794.0   
15     Dean Henderson    Sheffield Utd       GK      36.0      3240.0   
16    Wayne Hennessey   Crystal Palace       GK       3.0       270.0   
18         Bernd Leno          Arsenal       GK      30.0      2649.0   
19        Hugo Lloris        Tottenham       GK      21.0      1807.0   
20       David Martin         West Ham       GK       5.0       436.0   
21  Emiliano Martínez          Arsenal       GK       9.0       771.0   
22      Alex McCarthy      Southampton       GK      28.0      2520.0   
24        Simon Moore    Sheffield Utd       GK       2.0       180.0   
25       Ørjan Nyland      Aston Villa       GK       7.0       539.0   
26       Rui Patrício           Wolves       GK      38.0      3420.0   
27    Jordan Pickford          Everton       GK      38.0      3420.0   
28          Nick Pope          Burnley       GK      38.0      3420.0   
29     Aaron Ramsdale      Bournemouth       GK      37.0      3330.0   
30    Darren Randolph         West Ham       GK       2.0       180.0   
32            Roberto         West Ham       GK       8.0       687.0   
33        Mathew Ryan         Brighton       GK      38.0      3420.0   
34  Kasper Schmeichel   Leicester City       GK      38.0      3420.0   
35          Jed Steer      Aston Villa       GK       1.0         7.0   

    goals_against_per90_gk  save_pct  clean_sheets_pct  pens_allowed  \
0                     1.03     0.643              22.2           0.0   
1                     0.81     0.722              44.8           1.0   
2                     1.42     0.545              24.2           2.0   
3                     1.82     0.533              33.3           0.0   
4                     1.40     0.563              20.0           0.0   
5                     1.53     0.705              28.9           2.0   
6                     0.82     0.728              45.7           3.0   
7                     1.45     0.684              20.0           3.0   
10                    1.45     0.709              11.8           3.0   
11                    0.95     0.736              34.2           3.0   
12                    1.20     0.714              28.6           0.0   
13                    2.50     0.566              20.0           2.0   
14                    1.76     0.677              20.0           3.0   
15                    0.92     0.752              36.1           1.0   
16                    2.67     0.636               0.0           0.0   
18                    1.33     0.776              23.3           6.0   
19                    1.05     0.796              28.6           2.0   
20                    1.24     0.700              50.0           0.0   
21                    1.05     0.810              37.5           1.0   
22                    1.25     0.691              25.0           1.0   
24                    3.00     0.500               0.0           0.0   
25                    2.00     0.538              20.0           0.0   
26                    

In [10]:
keeper['ppm'] = keeper.apply(lambda row: row['points']/row['games_gk'],axis=1)
short_name = [" ".join(p_name.split()[1:]) if len(p_name.split()) != 1 else p_name for p_name in keeper['player']]
keeper.insert(13,'player_name',short_name)
keeper.head(10)

player            squad position  games_gk  minutes_gk  \
0              Adrián        Liverpool       GK      11.0       875.0   
1             Alisson        Liverpool       GK      29.0      2543.0   
2   Kepa Arrizabalaga          Chelsea       GK      33.0      2970.0   
3       Claudio Bravo  Manchester City       GK       4.0       347.0   
4     Willy Caballero          Chelsea       GK       5.0       450.0   
5     Martin Dúbravka    Newcastle Utd       GK      38.0      3420.0   
6             Ederson  Manchester City       GK      35.0      3071.0   
7    Łukasz Fabiański         West Ham       GK      25.0      2117.0   
10    Paulo Gazzaniga        Tottenham       GK      18.0      1613.0   
11       David de Gea   Manchester Utd       GK      38.0      3420.0   

    goals_against_per90_gk  save_pct  clean_sheets_pct  pens_allowed  \
0                     1.03     0.643              22.2           0.0   
1                     0.81     0.722              44.8           1.0   
2                     1.42     0.545              24.2           2.0   
3                     1.82     0.533              33.3           0.0   
4                     1.40     0.563              20.0           0.0   
5                     1.53     0.705              28.9           2.0   
6                     0.82     0.728              45.7           3.0   
7                     1.45     0.684              20.0           3.0   
10                    1.45     0.709              11.8           3.0   
11                    0.95     0.736              34.2           3.0   

    psxg_net_gk  cost  points       ppm   player_name  
0          -1.3   4.5      27  2.454545        Adrián  
1           0.8   6.0     122  4.206897       Alisson  
2          -9.6   5.0      90  2.727273  Arrizabalaga  
3          -3.0   4.5      11  2.750000         Bravo  
4          -0.4   5.0      13  2.600000     Caballero  
5          10.0   5.0     143  3.763158      Dúbravka  
6           2.4   6.0     133  3.800000       Ederson  
7          -2.0   5.0      83  3.320000     Fabiański  
10          0.2   4.5      58  3.222222     Gazzaniga  
11          2.5   5.5     143  3.763158        de Gea

In [11]:
keeper.to_csv('keeper_dash.csv',index=False)